In [2]:
import Ramsey_ExperimentV2
import estimator
import numpy as np

In [3]:
import Ramsey_ExperimentV2
import random
import importlib
from tqdm import tqdm

ramsey_experiment = importlib.reload(Ramsey_ExperimentV2)
RamseyExperiment = ramsey_experiment.RamseyExperiment
RamseyBatch = ramsey_experiment.RamseyBatch

h = lambda n, J, z: sum([J[i] * (z[i] - 1) * (z[(i + 1) % n] - 1) for i in range(n - 1)])
y_lornz = lambda a, x, w: np.pi * (2 * a / (a ** 2 + (x - w) ** 2) + 2 * a / (a ** 2 + (x + w) ** 2))
y_lornz_single = lambda a, x, w: (2 * a / ((a ** 2 + x ** 2) * 2 * np.pi))


def sinc(x_values):
    X = []
    for x in x_values:
        X.append((np.sin(2 * x * np.pi) / (2 * x * np.pi) if x != 0 else 1))
    return X


def fisher_inf(x, w, a):
    cos_wx_squared = np.cos(w * x) ** 2
    sin_wx_squared = np.sin(w * x) ** 2
    exp_ax = np.exp(a * x)

    term1 = x ** 2 * cos_wx_squared / (exp_ax + np.cos(w * x)) ** 2
    term2 = x ** 2 * sin_wx_squared / (exp_ax + np.cos(w * x)) ** 2

    return term1 + term2


def effective_hem(size, J, W):
    hem = np.zeros((2 ** size, 2 ** size))
    for i in range(2 ** size):
        binary = '{0:b}'.format(i).zfill(size)
        Z = [(-1) ** int(i) for i in binary]
        #Z.reverse()
        hem[i, i] = h(size, J, Z)
        hem[i, i] += sum([W[k] for k in range(size) if binary[k] == '1'])
    return hem

In [4]:
import numpy as np


def p(t, w, a):
    return 1 / 2 * (1 + np.cos(w * t) * np.exp(-a * t))


def pw(t, w, a):
    return - 1 / 2 * t * np.sin(w * t) * np.exp(-a * t)


def pww(t, w, a):
    return - 1 / 2 * t ** 2 * np.cos(w * t) * np.exp(-a * t)


def pa(t, w, a):
    return - 1 / 2 * t * np.cos(w * t) * np.exp(-a * t)


def paa(t, w, a):
    return 1 / 2 * t ** 2 * np.cos(w * t) * np.exp(-a * t)


def paw(t, w, a):
    return 1 / 2 * t ** 2 * np.sin(w * t) * np.exp(-a * t)


def fisher_matrix(t, w, a, shots):
    Iww = 0
    Iaa = 0
    Iwa = 0

    for i in range(len(t)):
        p0 = p(t[i], w, a)
        p1 = 1 - p0
        pw0 = pw(t[i], w, a)
        pw1 = -pw0
        pww0 = pww(t[i], w, a)
        pww1 = -pww0
        pa0 = pa(t[i], w, a)
        pa1 = -pa0
        paa0 = paa(t[i], w, a)
        paa1 = -paa0
        paw0 = paw(t[i], w, a)
        paw1 = -paw0

        if p0 == 0:
            p0 = 1e-10
        if p1 == 0:
            p1 = 1e-10

        # Iww -= shots[i] * (p0 * ((pww0 * p0 - pw0 ** 2) / p0 ** 2) + p1 * ((pww1 * p1 - pw1
        #                                                                     ** 2) / p1 ** 2))
        # Iaa -= shots[i] * (p0 * ((paa0 * p0 - pa0 ** 2) / p0 ** 2) + p1 * ((paa1 * p1 - pa1 ** 2) / p1 ** 2))
        # Iwa -= shots[i] * (p0 * ((paw0 * p0 - pw0 * pa0) / p0 ** 2) + p1 * ((paw1 * p1 - pw1 * pa1) / p1 ** 2))

        Iww += shots[i] * (p0 * ((pw0 ** 2) / p0 ** 2) + p1 * ((pw1 ** 2) / p1 ** 2))
        Iaa += shots[i] * (p0 * ((pa0 ** 2) / p0 ** 2) + p1 * ((pa1 ** 2) / p1 ** 2))
        Iwa += shots[i] * (p0 * ((pw0 * pa0) / p0 ** 2) + p1 * ((pw1 * pa1) / p1 ** 2))
    return np.array([[Iww, Iwa], [Iwa, Iaa]])



def lower_bound(f_mat):
    inv = np.linalg.inv(f_mat)
    w_var = np.sqrt(inv[0, 0])
    a_var = np.sqrt(inv[1, 1])

    #a_var = np.sqrt(inv[1, 1])
    return w_var, a_var



In [5]:
from scipy.optimize import minimize


def find_optimal_times(W, A, N, Total_Shots=1000):
    def model_func(params):
        t = params[0:N]
        shots = params[N:2 * N]
        Iww = [0] * len(W)
        Iaa = [0] * len(W)
        Iwa = [0] * len(W)

        for i in range(len(t)):
            for j in range(len(W)):
                w = W[j]
                a = A[j]
                p0 = p(t[i], w, a)
                p1 = 1 - p0
                pw0 = pw(t[i], w, a)
                pw1 = -pw0
                pww0 = pww(t[i], w, a)
                pww1 = -pww0
                pa0 = pa(t[i], w, a)
                pa1 = -pa0
                paa0 = paa(t[i], w, a)
                paa1 = -paa0
                paw0 = paw(t[i], w, a)
                paw1 = -paw0

                if p0 == 0:
                    p0 = 1e-10
                if p1 == 0:
                    p1 = 1e-10

                Iww[j] -= shots[i] * (p0 * ((pww0 * p0 - pw0 ** 2) / p0 ** 2) + p1 * ((pww1 * p1 - pw1
                                                                                    ** 2) / p1 ** 2))
                Iaa[j] -= shots[i] * (p0 * ((paa0 * p0 - pa0 ** 2) / p0 ** 2) + p1 * ((paa1 * p1 - pa1 ** 2) / p1 ** 2))
                Iwa[j] -= shots[i] * (p0 * ((paw0 * p0 - pw0 * pa0) / p0 ** 2) + p1 * ((paw1 * p1 - pw1 * pa1) / p1 ** 2))

        lower_w_bound = np.sum([10000 * Iaa[j] / (Iaa[j] * Iww[j] - Iwa[j] ** 2) for j in range(len(W))])
        lower_a_bound = np.sum([10000 * Iww[j] / (Iaa[j] * Iww[j] - Iwa[j] ** 2) for j in range(len(W))])
        return lower_w_bound + lower_a_bound

    def shots_constraint(params):
        return np.sum(params[N:2 * N]) - Total_Shots

    # Define the constraint as a dictionary
    cons = ({'type': 'eq', 'fun': shots_constraint})
    initial_guess = np.concatenate([np.random.uniform(0, np.pi, N), np.full(N, Total_Shots / N)])
    result = minimize(model_func, initial_guess, method='trust-constr', bounds=[(0, np.pi)] * N + [(0, 1000)] * N,
                      constraints=cons)
    return result.x


def process_result(result):
    times = result[0:N]
    shots = result[N:2 * N]

    grouped_times = []
    grouped_shots = []

    for i in range(len(times)):
        for j in range(len(grouped_times)):
            if abs(grouped_times[j] - times[i]) < 0.005:
                #grouped_times.append(times[i])
                grouped_shots[j] += shots[i]
                break
        else:
            grouped_times.append(times[i])
            grouped_shots.append(shots[i])

    paired = sorted(zip(grouped_times, grouped_shots))
    times = [x for x, _ in paired]
    shots = [int(round(y)) for _, y in paired]

    return times, shots


N = 10
Total_Shots = N * 500
w = [1]
a = [1]
result = find_optimal_times(w, a, N, Total_Shots)
optimal_times, optimal_shots = process_result(result)

regular_times = np.linspace(0, np.pi, N)
print("Optimal times = ", optimal_times)
print("Optimal shots = ", optimal_shots)
fishers = [fisher_matrix(optimal_times, w[i], a[i], [1000]*N) for i in range(len(w))]
print("var = ", np.sum([np.trace(fishers[i]) for i in range(len(w))]))

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_differentiable_functions.py:504: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_differentiable_functions.py:231: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


Optimal times =  [0.44394167647508664, 1.7846965547046325]
Optimal shots =  [2309, 2691]
var =  211.9237967897835


In [6]:
shots= np.logspace(2,6,num=6-2+1,base=10,dtype='int')
bounds = []
for s in shots:
    mat = fisher_matrix(optimal_times, w[0], a[0], [s]*N)
    bounds.append(lower_bound(mat)[1])
plt.plot(shots, bounds, label='2 times numeric')

plt.plot(shots, np.e * np.sqrt(a[0])/ np.sqrt(shots), '--', label='two quadratures analytical')

plt.legend()
plt.xlabel('Shots')
plt.ylabel('Error')
plt.xscale('log')
plt.yscale('log')

NameError: name 'plt' is not defined

In [ ]:


#optimal_times, optimal_shots = process_result(result)
print("Optimal times = ", optimal_times)
print("Optimal shots = ", optimal_shots)
print(sum(optimal_shots))
optimal_w_bound = np.sum(lower_bound(fisher_matrix(optimal_times, w, a, optimal_shots)))
regular_w_bound = np.sum(lower_bound(fisher_matrix(regular_times, w, a, N * [100])))
print("Optimal w bound = ", optimal_w_bound)
print("Regular w bound = ", regular_w_bound)

In [ ]:
##approximation:
from scipy.optimize import minimize


def find_optimal_times_app(W, A, t):
    def model_func(t):
        Iww = [0] * len(W)
        Iaa = [0] * len(W)
        Iwa = [0] * len(W)

        for i in range(len(t)):
            for j in range(len(W)):
                w = W[j]
                a = A[j]
                Iww[j] += t[i]**2 * np.sin(w * t[i])**2 * np.exp(-2 * a * t[i])
                Iaa[j] += t[i]**2 * np.cos(w * t[i])**2 * np.exp(-2 * a * t[i])
                Iwa[j] += t[i]**2 * np.sin(w * t[i]) * np.cos(w * t[i]) * np.exp(-2 * a * t[i])




        lower_w_bound = np.sum([Iaa[j] / (0.001 if Iaa[j] * Iww[j] - Iwa[j] ** 2 == 0 else Iaa[j] * Iww[j] - Iwa[j] ** 2) for j in range(len(W))])
        lower_a_bound = np.sum([Iww[j] / (0.001 if Iaa[j] * Iww[j] - Iwa[j] ** 2 == 0 else Iaa[j] * Iww[j] - Iwa[j] ** 2) for j in range(len(W))])
        return lower_w_bound + lower_a_bound


    initial_guess = np.random.uniform(0.1, np.pi, t)
    result  = minimize(model_func, initial_guess,method='trust-constr', bounds=[(0, np.pi)] * t)
    return result.x


t = 4
w = [1,5]
a = [1,5]
optimal_times = find_optimal_times_app(w, a, t)

print("Optimal times = ", optimal_times)
fishers = [fisher_matrix(optimal_times, w[i], a[i], [1000]*t) for i in range(len(w))]
print("var = ", np.sum([t*np.trace(np.linalg.inv(fishers[i])) / (w[i]) for i in range(len(w)) ]))

In [ ]:
from sympy import lambdify
## Normal approximation
from sympy import symbols, cos, sin, exp, sqrt
from scipy.optimize import minimize

# Define the symbols
t0, t1 = symbols('t0 t1', real=True, positive=True)

# Parameters
omega_value = gamma_value = w =a = 1

# Define the elements of the inverted matrix
I_00_expr = ((exp(2 * gamma_value * t1) * t0**2 * cos(omega_value * t0)**2 + exp(2 * gamma_value * t0) * t1**2 * cos(omega_value * t1)**2)
        * (1/sin(omega_value * (t0 - t1)))**2) / (t0**2 * t1**2)
I_11_expr = ((exp(2 * gamma_value * t1) * t0**2 * sin(omega_value * t0)**2 + exp(2 * gamma_value * t0) * t1**2 * sin(omega_value * t1)**2)
        * (1/sin(omega_value * (t0 - t1)))**2) / (t0**2 * t1**2)


# Define the function we want to find the minimum for
f_expr = sqrt(I_00_expr) + sqrt(I_11_expr)
#f_expr = sqrt(I_00_expr)
# Convert the symbolic expression to a numerical function
f_numeric = lambdify((t0, t1), f_expr, modules=['numpy'])

# Define a function to be minimized
def f_to_minimize(x):
    # x is a vector where x[0] = t0 and x[1] = t1
    return f_numeric(x[0], x[1])

# Initial guess (we avoid starting at 0 to prevent division by zero)
initial_guess = [0.1, 1]

# Perform the minimization
result = minimize(f_to_minimize, initial_guess, bounds=[(0.1, 5), (0.1, 5)])

# Extract the results
t0_min, t1_min = result.x
min_value = result.fun

print(t0_min, t1_min, min_value)
print("approximated_bound = ",np.sum(lower_bound(fisher_matrix([t0_min,t1_min], w, a, [2500,2500]))))



In [ ]:
from sympy import symbols, cos, sin, exp, csc, sqrt, diff, solve

# Define the symbols
t0, t1, omega, gamma = symbols('t0 t1 omega gamma', real=True, positive=True)

# Define the elements of the inverted matrix
I_00 = ((exp(-2 * gamma * t1) * t0**2 * cos(omega * t0)**2 + exp(-2 * gamma * t0) * t1**2 * cos(omega * t1)**2)
        * csc(omega * (t0 - t1))**2) / (t0**2 * t1**2)
I_11 = ((exp(-2 * gamma * t1) * t0**2 * sin(omega * t0)**2 + exp(-2 * gamma * t0) * t1**2 * sin(omega * t1)**2)
        * csc(omega * (t0 - t1))**2) / (t0**2 * t1**2)

# Define the function we want to minimize
f = sqrt(I_00) + sqrt(I_11)

# Compute the derivative of the function with respect to both t0 and t1
df_dt0 = diff(f, t0)
df_dt1 = diff(f, t1)

# Find the critical points by setting the derivatives to zero
critical_points = solve((df_dt0, df_dt1), (t0, t1))

critical_points


In [ ]:
VarA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
w=1
a = 1.5
t = np.linspace(0, 1*np.pi, 6000)  # time vector
# Defining f1, f2, f3
f1 = (np.cos(w*t) + 1j*np.sin(w*t)) * np.exp(-a*t)
f2 = (np.cos(1.5*w*t) + 1j*np.sin(1.5*w*t)) * np.exp(-a*t)
f3 = (np.cos(3.5*w*t) + 1j*np.sin(3.5*w*t)) * np.exp(-a*t)

# Updated Complex Function
real_part = (np.cos(w*t) + np.cos(1.5*w*t) + np.cos(3.5*w*t)) * np.exp(-a*t)
imaginary_part = (np.sin(w*t) + np.sin(1.5*w*t) + np.sin(3.5*w*t)) * np.exp(-a*t)



# Logarithm
real_part = np.log(real_part)
imaginary_part = np.log(imaginary_part)
f1 = np.log(f1)
f2 = np.log(f2)
f3 = np.log(f3)

# Plot
plt.figure(figsize=(8, 6))
plt.plot(real_part, imaginary_part, label=r'$\cos(\omega t) + \cos(1.5\omega t) + \cos(3.5\omega t) + i(\sin(\omega t) + \sin(1.5\omega t) + \sin(3.5\omega t))$')
plt.title('Complex Plane Plot')
plt.xlabel('Real Part')
plt.ylabel('Imaginary Part')
plt.legend()
plt.grid(True)
plt.show()


# Updated Plot
plt.figure(figsize=(8, 6))
plt.plot(real_part, imaginary_part, label=r'Total: $\cos(\omega t) + \cos(1.5\omega t) + \cos(3.5\omega t) + i(\sin(\omega t) + \sin(1.5\omega t) + \sin(3.5\omega t))$', zorder=1)
plt.plot(f1.real, f1.imag, '--', label=r'$f_1 = \cos(\omega t) + i\sin(\omega t)$', zorder=2)
plt.plot(f2.real, f2.imag, '--', label=r'$f_2 = \cos(1.5\omega t) + i\sin(1.5\omega t)$', zorder=2)
plt.plot(f3.real, f3.imag, '--', label=r'$f_3 = \cos(3.5\omega t) + i\sin(3.5\omega t)$', zorder=2)
plt.title('Complex Plane Plot with $f_1$, $f_2$, and $f_3$')
plt.xlabel('Real Part')
plt.ylabel('Imaginary Part')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# 3D Spiral
ax.plot(f1.real, f1.imag, t, label='Spiral Shape')
ax.set_title('3D Plot of $\sin(\omega t) + \cos(\omega t)$ Showing Spiral Shape')
ax.set_xlabel('Real Part ($\sin(\omega t)$)')
ax.set_ylabel('Imaginary Part ($\cos(\omega t)$)')
ax.set_zlabel('Time ($t$)')
ax.legend()

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# 3D Spiral
ax.plot(f2.real, f2.imag, t, label='Spiral Shape')
ax.set_title('3D Plot of $\sin(\omega t) + \cos(\omega t)$ Showing Spiral Shape')
ax.set_xlabel('Real Part ($\sin(\omega t)$)')
ax.set_ylabel('Imaginary Part ($\cos(\omega t)$)')
ax.set_zlabel('Time ($t$)')
ax.legend()

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# 3D Spiral
ax.plot(real_part, imaginary_part, t, label='Spiral Shape')
ax.set_title('3D Plot of $\sin(\omega t) + \cos(\omega t)$ Showing Spiral Shape')
ax.set_xlabel('Real Part ($\sin(\omega t)$)')
ax.set_ylabel('Imaginary Part ($\cos(\omega t)$)')
ax.set_zlabel('Time ($t$)')
ax.legend()